# TODO
- Hinzufügen von Isohypsen ohne Dupuit-Annahme in den Plot, damit man die Abweichung zwischen Strömungsvektoren mit und ohne Dupuit-Annahme interaktiv vergleich kann
- Referenzierung zum Beispiel der Grabenströmung
- Ein Standard-Layout nach Wagemann

In [4]:
# Initialize librarys
from scipy.special import erfc, erf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
import json
from ipywidgets import *
import sys

In [5]:
# Choose language for plot and assign to strings
#
# The language dictionary is a preliminary idea 
# to make plots usable across different unis
#
english = False    

if english:
    label = language["Berechnetes Druckpotential"]
    ylabel = language["Druckpotential"] + " (m)"
    title_ax1 = language["Potentialhöhe für die 1D ungespannte Grundwasserströmung"]
    title_ax2 = language["Lineare Regression"]
    gwn = language["GWN"]
else:
    label = "Berechnetes Druckpotential"
    ylabel = "Druckpotential" + " (m)"
    title_ax1 = "Potentialhöhe für die 1D ungespannte Grundwasserströmung"
    title_ax2 = "Lineare Regression"
    gwn = "GWN"

# Dupuit-Annahme

Im ungespannten Grundwasserleiter können viele Fragestellungen unter Verwendungen einer bestimmten Annahme deutlich vereinfacht werden. Es handelt sich um die sogenannte Dupuit-Annahme.

### Was sagt die Dupuit-Annahme?
Die Dupuit-Annahme stellt zwei Bedingungen:
- Grundwasser strömt nur horizontal
- Grundwasser strömt mit gleicher Geschwindigkeit über die Höhe


Strömungsvektoren (schwarzen Pfeile), die nicht horizontal verlaufen, sind demnach nicht erlaubt:
![](crosssection_arrows_incorrect1.png "Querschnitt eines ungespannten Flusses mit Dupuit-Annahme und falschen Strömungsvektoren")

Genauso wenig dürfen die Strömungsvektoren unterschiedlich lang über die Höhe sein:
![](crosssection_arrows_incorrect2.png "Querschnitt eines ungespannten Flusses mit Dupuit-Annahme und falschen Strömungsvektoren")

Strömungsvektoren sind also horizontal und gleich lang:

![](crosssection_arrows_correct.png "Querschnitt eines ungespannten Flusses mit Dupuit-Annahme und richtigen Strömungsvektoren")

### Welche Möglichkeiten bietet die Dupuit-Annahme?

Die beiden Bedingungen der Dupuit-Annahme ermöglichen, komplizierte Sachverhalte in 2D für den ungespannten Grundwasserleiter auf 1D zu reduzieren. Viele Fragestellungen werden dadurch  erst analytisch lösbar. 

So zum Beispiel die [Grabenströmung](). Die Gleichung $h(x)=\sqrt{H_l^2-\frac{H_l^2-H_r^2}{L}x+\frac{q_n}{K}x(L-x)}$ kann nur mit der Dupuit-Annahme hergeleitet werden.

Gleichermaßen benutzen Modelle, die nicht über die Höhe aufgelöst sind (in Modflow: die nur aus einem Layer bestehen) ebenfalls implizit die Dupuit-Annahme. Denn auch hier ist die Strömungsgeschwindigkeit horizontal und über die Höhe gleich.

Wichtig ist, bei allen diesen analytischen Lösungen und Modellen, die mit der Dupuit-Annahme einhergehenden Bedingungen zu kennen. Diese müssen stets hinterfragt und dann gerechtfertigt werden.

### Welche Grenzen hat die Dupuit-Annahme?
Die Bedingungen der Dupuit-Annahme stellen nur eine Vereinfachung dar und sind dadurch nie richtig. Der Fehler ist immer mehr vernachlässigbar, je mehr die geforderten Bedingungen einer horizontalen und über die Höhe gleichen Grundwasserströmung vorliegen. Das ist dann der Fall, wenn...

- ...der hydraulische Gradiente gering ist
- ...die Mächtigkeit des Grundwasserleiters gegenüber seiner Ausdehnung gering ist
- ...der Grundwasserleiter homogen und isotrop ist

Schau Dir das nachfolgende Beispiel der Grabenströmung an. Was passiert, wenn der hydraulische Gradient mehr und mehr vergrößert wird? Vergleiche die wirklichen Strömungsvektoren mit den Strömungsvektoren der Dupuit-Annahme!

In [3]:
def calculate_head(R, hr, hl, K, L, x):
    """
    Calculate the head after Depuit-Forchheimer-equation
    
    Keyword Arguments:
    R -- Recharge, m/s
    hr -- head at x=0, m
    hl -- head at x=L, m
    K -- hydraulic conductivity, m/s
    L -- distance to hl, m
    x -- points to get the head for, m
    """
    h=(hl**2-(hl**2-hr**2)/L*x+(R/K*x*(L-x)))**0.5
    return h

def poly_isohypse():
    """
    Get the function for poly fit
    """
    return a * y**3 + b * y**2 + c

def get_poly_isohypse(hr, hl, K, L, x):
    h_iso = (hr - hl) * x / L + hl
    x_top = h_iso**2 / (hl**2 - (hl**2 - hr**2) / L)
    slope = - 0.5 * (hl**2 - (hl**2 - hr**2) * x / L)**(-0.5) * (hl**2 - hr**2) / L
    slope_inv = 1 / slope
    mat = np.array([[3 * h_iso**2, 2 * h_iso],[h_iso**3, h_iso**2]])
    res = np.array([[slope_inv - x],[x_top - x]])
    a, b = np.linalg.solve(mat, res)
    def poly(y):
        return a * y**3 + b * y**2 + x
    return poly

# Definition of the function
def head(hl, hr, L, R, K,y_scale):
    """
    Plot the head and correlation
    
    Keyword Arguments:
    R -- Recharge, mm/a
    hr -- head at x=0, m
    hl -- head at x=L, m
    K -- hydraulic conductivity, m/s
    L -- distance to hl, m
    y_scale -- scale of y, -
    """
    
    # Transform recharge units from mm/a to m/s
    R=R/1000/365.25/86400
    
    # Calculate head
    x = np.arange(0, L,L/1000)
    #global h
    h = calculate_head(R, hr, hl, K, L, x)
    
    # Prepare figure and subplots
    fig = plt.figure(figsize=(9,6))
    ax = fig.add_subplot(1, 1, 1)
    
    # Plot the head
    ax.plot(x,h, label=label)
    ax.set(xlabel='x (m)', ylabel=ylabel,title=title_ax1)
    ax.fill_between(x,0,h, facecolor='lightblue')
    
    # Plot the CH-boundaries
    ax.vlines(0, 0, hl, linewidth = 10, color='b')
    ax.vlines(L, 0, hr, linewidth = 10, color='b')
    
    # Make Water-Triangle
    y_range = abs((hl*(1-y_scale/100))-(hr*(1+y_scale/100)))
    h_arrow = (hl**2-(hl**2-hr**2)/L*(L*0.96)+(R/K*(L*0.96)*(L-(L*0.96))))**0.5  #water level at arrow
    ax.arrow(L*0.96,(h_arrow+(h_arrow*0.004)), 0, -0.01, fc="k", ec="k", head_width=(L*0.015), head_length=(y_range*0.03))
    #ax.hlines(y= h_arrow-(h_arrow*0.0005), xmin=L*0.95, xmax=L*0.97, colors='blue')   
    ax.hlines(y= h_arrow-(y_range*0.01), xmin=L*0.95, xmax=L*0.97, colors='blue') 
    #ax.hlines(y= h_arrow-(h_arrow*0.001), xmin=L*0.955, xmax=L*0.965, colors='blue')
    ax.hlines(y= h_arrow-(y_range*0.015), xmin=L*0.955, xmax=L*0.965, colors='blue')
    
    # Add arrows for recharge
    if R != 0:
        head_length=(R*1000*0.0005 * (86400*365.25))*y_range # DIVISION STATT MULTIPLIKATION?
        h_rch1 = (hl**2-(hl**2-hr**2)/L*(L*0.25)+(R/K*(L*0.25)*(L-(L*0.25))))**0.5  #water level at arrow for Recharge Posotion 1
        ax.arrow(L*0.25,(h_rch1+head_length), 0, -0.01, fc="k", ec="k", head_width=(L*0.03), head_length=head_length)
        h_rch2 = (hl**2-(hl**2-hr**2)/L*(L*0.50)+(R/K*(L*0.50)*(L-(L*0.50))))**0.5  #water level at arrow for Recharge Postition 2
        ax.arrow(L*0.50,(h_rch2+head_length), 0, -0.01, fc="k", ec="k", head_width=(L*0.03), head_length=head_length)
        h_rch3 = (hl**2-(hl**2-hr**2)/L*(L*0.75)+(R/K*(L*0.75)*(L-(L*0.75))))**0.5  #water level at arrow for Recharge Position 3
        ax.arrow(L*0.75,(h_rch3+head_length), 0, -0.01, fc="k", ec="k", head_width=(L*0.03), head_length=head_length)

    # Add Grundwasserscheide
    max_y = max(h)
    max_x = x[h.argmax()]
    R_min_ms=K*abs(hl**2-hr**2)/L**2
    if R>R_min_ms:
        plt.vlines(max_x,0,max_y, color="r")
        
    
    # Set y min und max
    y_min = hl*(1-y_scale/100)
    y_max = hr*(1+y_scale/100)
    
    # Add flow arrows
    grad_h = np.diff(h)
    num_arows = 10

    max_arrow_length = 100
    start_arrow = 0 + 30 + max_arrow_length
    end_arrow = L - max_arrow_length
    steps = (end_arrow - start_arrow) / num_arows
    numeric_x = np.concatenate((np.arange(max_x,start_arrow, -steps), np.arange(max_x, end_arrow, steps)))
    numeric_x = np.delete(numeric_x, np.argwhere(numeric_x == max_x))
    
    sample_x = [np.absolute(x-x_val).argmin() for x_val in numeric_x]
    max_grad = np.max(abs(grad_h[sample_x]))
    
    
    puffer = (y_max - y_min) * 0.02
    arrow_head_width = (np.min(h[sample_x]) - y_min) * 0.6 / num_arows
    for idx in sample_x:
        arrow_length = max_arrow_length * grad_h[idx] / max_grad
        x_pos = x[idx]
        y_positions = np.linspace(y_min + puffer, h[idx]  - puffer, num_arows)
        for y_pos in y_positions:
            #print(f"plot arrow at {x_pos}, {y_pos}")
            plt.arrow(x_pos, y_pos, -arrow_length, 0, width=arrow_head_width/2, 
                      head_width=arrow_head_width,head_length=y_scale*3.,
                      length_includes_head=True)
    #if plot_real_flow:
        #for sample_x
    #poly = get_poly_isohypse(hr, hl, K, L, x[sample_x[0]])
    #y_positions = np.linspace(y_min + puffer, h[sample_x[0]]  - puffer, num_arows)
    #plt.scatter(poly(y_positions), y_positions)
    
    plt.ylim(y_min, y_max)
    plt.xlim(-50,L+50)
    plt.text(L, (hr*(1+y_scale/100))-0.1*y_range, gwn+r': {:.3e} m/s '.format(R), horizontalalignment='right', bbox=dict(boxstyle="square", facecolor='azure'),fontsize=12)
    ax.grid()
    
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()


In [43]:
# Start interactive plot

interactive_plot = interact(head,
         y_scale = widgets.BoundedFloatText(value=5, min=1, max=100, step=1, description='Skal. y-Achse:', disabled=False),
         hl=widgets.BoundedFloatText(value=100, min=0, max=1000, step=1, description='H_l:', disabled=False),
         hr=widgets.BoundedFloatText(value=102, min=0, max=1000, step=1, description='H_r:', disabled=False),
         L= widgets.BoundedFloatText(value=2500,min=0, max=20000,step=100, description='L:' , disabled=False),
         R=widgets.FloatSlider(value=0,min=-500, max=500, step=10,description=gwn+":", disabled=False),
         K=widgets.FloatLogSlider(value=0.0001,base=10,min=-6, max=-2, step=0.1,description='K:',readout=True,readout_format='.2e'))


interactive(children=(BoundedFloatText(value=100.0, description='H_l:', max=1000.0, step=1.0), BoundedFloatTex…